In [2]:
from library import *

Library imports successful


### MVP Testing:

    bm_terrorist_success_rate.joblib
    rfc_terrorist_success_rate.joblib
    lg_terrorist_success_rate.joblib

In [3]:
input_data = pd.DataFrame([{
    'Year': 2025,
    'Month': 1,                  # maybe a low-activity month
    'Country': 'Switzerland',    # likely low-risk country
    'Region': 'Europe',
    'Attack_Type': 'Theft',      # less severe type
    'Weapon_Type': 'None',       # no weapon
    'ismilitary': 1,             # not targeting military
    'Nationality': 'Unknown'
}])

input_data1 = pd.DataFrame([{
    'Year': 2025,
    'Month': 12,
    'Country': 'Afghanistan',
    'Region': 'South Asia',
    'Attack_Type': 'Bombing',
    'Weapon_Type': 'Explosives',
    'ismilitary': 1,
    'Nationality': 'Local'
}])

# Likely successful attack (high-risk scenario)
input_data2 = pd.DataFrame([{
    'Year': 2025,
    'Month': 7,                 # mid-year, higher activity
    'Country': 'Iraq',
    'Region': 'Middle East',
    'Attack_Type': 'Armed Assault',
    'Weapon_Type': 'Firearms',
    'ismilitary': 1,            # targeting military
    'Nationality': 'Local'
}])

# Likely failed attack (low-risk scenario)
input_data3 = pd.DataFrame([{
    'Year': 2025,
    'Month': 2,                 # low-activity month
    'Country': 'Canada',
    'Region': 'North America',
    'Attack_Type': 'Vandalism',
    'Weapon_Type': 'None',
    'ismilitary': 0,            # civilian target
    'Nationality': 'Unknown'
}])


In [4]:
saved_data = joblib.load('../model/lg_terrorist_success_rate.joblib')
model = saved_data['model']
metadata = saved_data['metadata']

test = input_data

prediction = model.predict(test)
confidence = model.predict_proba(test)
print(prediction)
print("Confidence probabilities:", confidence[0])

# Human-readable interpretation
interpretation = "Likely to succeed" if prediction[0] == 1 else "Likely to fail"
confidence_score = confidence[0][prediction[0]]
print(f"Prediction: {interpretation} ({confidence_score*100:.2f} Confidence)")


[0]
Confidence probabilities: [0.72626991 0.27373009]
Prediction: Likely to fail (72.63 Confidence)
